# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# Review the collections in our database
print("\nCollections in 'uk_food' database:")
print(db.list_collection_names())

In [ ]:
# Assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"\nNumber of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
result = establishments.find(query)
print("\nFirst document with a hygiene score of 20:")
pprint(result[0])


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_20_df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(hygiene_20_df)}")

# Display the first 10 rows of the DataFrame
hygiene_20_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}


# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"\nNumber of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint
result = establishments.find(query)
print("\nFirst document with a RatingValue >= 4 in London:")
pprint(result[0])


In [ ]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(london_rating_df)}")

# Display the first 10 rows of the DataFrame
london_rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})

# Define the degree search range
degree_search = 0.01

latitude = float(penang_flavours['geocode']['latitude'])
longitude = float(penang_flavours['geocode']['longitude'])


# Construct the query
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

# Define the sort order
sort = [("scores.Hygiene", 1)]

# Limit the results to top 5
limit = 5

# Print the results
print("\nTop 5 establishments with a RatingValue of 5, sorted by lowest hygiene score, near 'Penang Flavours':")
for doc in result:
    pprint(doc)

In [ ]:
# Convert result to Pandas DataFrame
top_5_df = pd.DataFrame(list(result))
top_5_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Define the aggregation pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the pipeline
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"\nNumber of documents in the result: {len(results)}")

# Print the first 10 results
print("\nFirst 10 results:")
for result in results[:10]:
    pprint(result)


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_0_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(hygiene_0_df)}")

# Display the first 10 rows of the DataFrame
hygiene_0_df.head(10)